In [9]:
import sys
import time
import json
from pathlib import Path
from bedrock_agentcore_starter_toolkit import Runtime
import boto3
from boto3.session import Session
import os
import shutil

In [2]:
boto_session = Session()
region_name = boto3.Session().region_name
#SSM을 활용해서 Bedrock AgenticCore ARN 가져오기
ssm_client = boto_session.client('ssm', region_name=region_name)

In [3]:
def wait_for_deletion(runtime_id, client):
    """Runtime 삭제 완료 대기"""
    print("⏳ Runtime 삭제 완료 대기 중...")
    
    for i in range(30):
        try:
            client.get_agent_runtime(agentRuntimeId=runtime_id)
            print(f"📊 삭제 진행 중... ({i*30}초 경과)")
            time.sleep(30)
        except client.exceptions.ResourceNotFoundException:
            print("✅ Runtime 삭제 완료!")
            return
        except Exception as e:
            print(f"⚠️ 상태 확인 오류: {e}")
            time.sleep(30)
    
    print("⚠️ 삭제 완료 확인 시간 초과 (15분)")

In [4]:
try:
    agent_arn_response = ssm_client.get_parameter(
            Name="/basic_server/runtime_iam/agent_arn"
        )
    role_arn = agent_arn_response['Parameter']['Value']
    runtime_id = role_arn.split('/')[-1]
    client = boto3.client('bedrock-agentcore-control', region_name=region_name)
    client.delete_agent_runtime(agentRuntimeId=runtime_id)
    print(f"✅ Runtime 삭제 시작: {runtime_id} (리전: {region_name})")
    
    # 삭제 완료 대기
    wait_for_deletion(runtime_id, client)
except Exception as e:
    print(f"⚠️ Runtime 삭제 실패: {e}")


⚠️ Runtime 삭제 실패: An error occurred (ConflictException) when calling the DeleteAgentRuntime operation: Cannot update agent arn:aws:bedrock-agentcore:us-west-2:967607059098:runtime/basic_Agentic_Core-gymysLAdvx while it's DELETING. Please try again when the agent is ready.


In [6]:
try:
    ecr_url_response = ssm_client.get_parameter(
                Name="/basic_server/runtime_iam/ecr_repository_uri"
            )
    ecr_uri = ecr_url_response['Parameter']['Value']
    
    # URI에서 repository name 추출
    # 예: 123456789012.dkr.ecr.us-west-2.amazonaws.com/my-repo:latest
    repo_name = ecr_uri.split('/')[-1].split(':')[0]
    
    # ECR 삭제
    ecr = boto3.client('ecr', region_name=region_name)
    ecr.delete_repository(repositoryName=repo_name, force=True)
    print(f"✅ ECR 삭제: {repo_name} (리전: {region_name})")
except Exception as e:
    print(f"⚠️ ECR 삭제 실패: {e}")

⚠️ ECR 삭제 실패: An error occurred (RepositoryNotFoundException) when calling the DeleteRepository operation: The repository with name 'bedrock-agentcore-basic_agentic_core' does not exist in the registry with id '967607059098'


In [7]:
try:
    iam_role_response = ssm_client.get_parameter(
        Name="/basic_server/runtime_iam/execution_role_arn"
    )
    role_arn = iam_role_response['Parameter']['Value']
    
    # ARN에서 role name 추출
    # 예: arn:aws:iam::123456789012:role/my-role-name
    role_name = role_arn.split('/')[-1]
    
    iam = boto3.client('iam')
    
    # 인라인 정책 삭제
    policies = iam.list_role_policies(RoleName=role_name)
    for policy in policies['PolicyNames']:
        iam.delete_role_policy(RoleName=role_name, PolicyName=policy)
    
    # 관리형 정책 분리
    attached_policies = iam.list_attached_role_policies(RoleName=role_name)
    for policy in attached_policies['AttachedPolicies']:
        iam.detach_role_policy(RoleName=role_name, PolicyArn=policy['PolicyArn'])
    
    # 역할 삭제
    iam.delete_role(RoleName=role_name)
    print(f"✅ IAM 역할 삭제: {role_name}")
except Exception as e:
    print(f"⚠️ ECR 삭제 실패: {e}")

✅ IAM 역할 삭제: agentcore-runtime-basic_Agentic_Core-role


In [10]:
items_to_delete = [
        '.bedrock_agentcore.yaml',
        '.dockerignore',
         'Dockerfile'
    ]
    
for item in items_to_delete:
    try:
        if os.path.isfile(item):
            os.remove(item)
            print(f"✅ 파일 삭제: {item}")
        elif os.path.isdir(item):
            shutil.rmtree(item)
            print(f"✅ 디렉토리 삭제: {item}")
        else:
            print(f"⚠️ 존재하지 않음: {item}")
    except Exception as e:
        print(f"❌ 삭제 실패 {item}: {e}")

⚠️ 존재하지 않음: .bedrock_agentcore.yaml
⚠️ 존재하지 않음: .dockerignore
✅ 파일 삭제: Dockerfile
